## **Setting up Development Environment**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
% cd /content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1

/content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1


In [3]:
! pip install -U -q transformers
! pip install -U -q sentencepiece

     |████████████████████████████████| 3.4 MB 5.1 MB/s 
     |████████████████████████████████| 67 kB 4.1 MB/s 
     |████████████████████████████████| 596 kB 74.6 MB/s 
     |████████████████████████████████| 3.3 MB 40.9 MB/s 
     |████████████████████████████████| 895 kB 55.7 MB/s 
     |████████████████████████████████| 1.2 MB 5.2 MB/s 


## **Importing Libraries**

In [4]:
import pandas as pd

import numpy as np
import re

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from transformers import AutoTokenizer, AutoModel

import pickle

## **Loading Data**

In [5]:
dataframe = pd.read_csv("Dataset/tamil_offensive_train.tsv", sep='\t')

In [6]:
dataframe.head()

,id,text,category
0,tam1,திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...,NOT
1,tam2,இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...,OFF
2,tam3,மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...,NOT
3,tam4,மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......,OFF
4,tam5,only for விஜய் சேதுபதி and STR,NOT


In [7]:
dataframe.describe()

,id,text,category
count,5880,5880,5880
unique,5880,5812,3
top,tam4682,வந்தா ராஜாவா தான் வருவேன்........ ராஜாவா வந்தா...,NOT
freq,1,5,4724


In [8]:
dataframe['category'].unique()

array(['NOT', 'OFF', 'not-Tamil'], dtype=object)

In [9]:
dataframe.shape

(5880, 3)

In [10]:
dataframe = dataframe.loc[dataframe['category'] != 'not-Tamil']
dataframe['category'].unique()

array(['NOT', 'OFF'], dtype=object)

In [11]:
dataframe.shape

(5877, 3)

In [12]:
text = dataframe['text']

In [13]:
text

0       திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...
1       இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...
2       மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...
3       மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......
4                          only for விஜய் சேதுபதி and STR
                              ...                        
5875    இப்போதான் ஜாதி பிரச்சினை இல்லாம இருக்கு இப்போ ...
5876    இது ஒரு சாதி ரீதியான படம் இல்லை தான் செல்லும் ...
5877    கதை தெரியவில்லை  இருந்தாலும் மாறுபட்ட சிந்தனை ...
5878    அட பரவால்லையே 8.5 டிஸ்லைக்ஸ் போட்ருக்கானுங்க.....
5879    உதயநிதிக்கு நடிப்பு வரலனு சொல்ரவங்க லைக் போடுங...
Name: text, Length: 5877, dtype: object

In [14]:
label = dataframe['category']

In [15]:
label

0       NOT
1       OFF
2       NOT
3       OFF
4       NOT
       ... 
5875    NOT
5876    NOT
5877    NOT
5878    OFF
5879    NOT
Name: category, Length: 5877, dtype: object

## **Data Preprocessing**

In [16]:
le = LabelEncoder()
label = le.fit_transform(label)
label

array([0, 1, 0, ..., 0, 1, 0])

In [17]:
text = text.str.replace(r"[+/#@&*$%:]",'')
text = text.to_numpy()
text

array(['திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற வாழ்த்துக்கள்',
       'இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இதை தியேட்டர் ல போய் பார்க்கனுமா.. .',
       'மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வெற்றிபெற வாழ்த்துகிறேன்',
       ...,
       'கதை தெரியவில்லை  இருந்தாலும் மாறுபட்ட சிந்தனை வாழ்த்துக்கள்  பறையர் சார்பாக  நன்றி மோகன் ஜி',
       'அட பரவால்லையே 8.5 டிஸ்லைக்ஸ் போட்ருக்கானுங்க.. பரவால வயிறு எறிஞ்சி சாவுங்கடா சாவுங்கடா சாக்காடை நாய்ங்களா..இன்னும் நிறைய டிஸ்லைக்ஸ் எதிர் பாக்குறன்..',
       'உதயநிதிக்கு நடிப்பு வரலனு சொல்ரவங்க லைக் போடுங்கள் ...'],
      dtype=object)

In [18]:
with open('./TaMillion/tamil_stopwords.txt', encoding = 'utf-8') as f:
    tamil_stopwords = f.readlines()
    for i in range(len(tamil_stopwords)):
        tamil_stopwords[i] = re.sub('\n','',tamil_stopwords[i])
stopwords = tamil_stopwords

# Function for removing stop words
def stopwords_remove(text):
    # Appending words which are not stop words
    text = text.split(" ")
    removed = [s for s in text if s not in stopwords]  
    return removed

In [19]:
text = [stopwords_remove(s) for s in text]
for i in range(len(text)):
  text[i] = (" ".join(text[i]))

## **Feature Extraction**

In [20]:
# Load Transformer Model

tokenizer = AutoTokenizer.from_pretrained("monsoon-nlp/tamillion")
model = AutoModel.from_pretrained("monsoon-nlp/tamillion")

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/736 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/817k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at monsoon-nlp/tamillion were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
# Sample Output

tokenized_input = tokenizer(
        text[0],
        padding=True,
        truncation=False,
        return_tensors='pt'
    )

sample_output = model(**tokenized_input)

In [22]:
print(tokenized_input)

{'input_ids': tensor([[    3, 12190,  9946, 12372,  6651,  5052,  3322,  2599, 19040,  5026,
             4]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [23]:
print(sample_output)

BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[[-8.7158e-04, -1.5213e-04,  1.6477e+00,  ..., -1.5470e-03,
          -2.9372e-05,  9.1286e-05],
         [-8.7158e-04, -1.5213e-04,  1.6477e+00,  ..., -1.5470e-03,
          -2.9372e-05,  9.1286e-05],
         [-8.7158e-04, -1.5213e-04,  1.6477e+00,  ..., -1.5470e-03,
          -2.9372e-05,  9.1286e-05],
         ...,
         [-8.7158e-04, -1.5213e-04,  1.6477e+00,  ..., -1.5470e-03,
          -2.9372e-05,  9.1286e-05],
         [-8.7158e-04, -1.5213e-04,  1.6477e+00,  ..., -1.5470e-03,
          -2.9372e-05,  9.1286e-05],
         [-8.7158e-04, -1.5213e-04,  1.6477e+00,  ..., -1.5470e-03,
          -2.9372e-05,  9.1286e-05]]], grad_fn=<NativeLayerNormBackward0>), past_key_values=None, hidden_states=None, attentions=None, cross_attentions=None)


In [24]:
print(sample_output.last_hidden_state.cpu().detach().numpy().shape)

(1, 11, 768)


In [25]:
def max_pool(model_output):
    pooled_output = []
    for i in range(len(model_output[0][0])):
        max = model_output[0][0][i]
        for j in range(len(model_output[0])):
            if max < model_output[0][j][i]:
                max = model_output[0][j][i]
        pooled_output.append(max)
    pooled_output = np.array(pooled_output)
    pooled_output = pooled_output.reshape(1, len(pooled_output))
    return pooled_output

In [26]:
# Tokenizing Input Data

input = []

for i in text:
    tokenized_input = tokenizer(
        i,
        padding=True,
        truncation=False,
        return_tensors='pt'
    )
    input.append(tokenized_input)

In [27]:
# Embedding the Input Data

output = []

j = 1

for i in range(len(input)):
    
    model_output = model(**input[i])
    model_output = model_output.last_hidden_state.cpu().detach().numpy()
    output.append(max_pool(model_output))
    
    # Saving Embedded Input Data to Disk

    if ((i + 1) % 200) == 0:
        
        file_name = "./TaMillion/TaMillion-Stopword-NOStem-Embedded-TrainData/output" + str(j) + ".pickle"
        
        with open(file_name, "wb") as fp:   
            # Pickling
            pickle.dump(output, fp)
        
        print(file_name + " done")
        output = []
        j += 1
        
file_name = "./TaMillion/TaMillion-Stopword-NOStem-Embedded-TrainData/output" + str(j) + ".pickle"

with open(file_name, "wb") as fp:   
    # Pickling
    pickle.dump(output, fp)

print(file_name + " done")
output = []

./TaMillion/TaMillion-Stopword-NOStem-Embedded-TrainData/output1.pickle done
./TaMillion/TaMillion-Stopword-NOStem-Embedded-TrainData/output2.pickle done
./TaMillion/TaMillion-Stopword-NOStem-Embedded-TrainData/output3.pickle done
./TaMillion/TaMillion-Stopword-NOStem-Embedded-TrainData/output4.pickle done
./TaMillion/TaMillion-Stopword-NOStem-Embedded-TrainData/output5.pickle done
./TaMillion/TaMillion-Stopword-NOStem-Embedded-TrainData/output6.pickle done
./TaMillion/TaMillion-Stopword-NOStem-Embedded-TrainData/output7.pickle done
./TaMillion/TaMillion-Stopword-NOStem-Embedded-TrainData/output8.pickle done
./TaMillion/TaMillion-Stopword-NOStem-Embedded-TrainData/output9.pickle done
./TaMillion/TaMillion-Stopword-NOStem-Embedded-TrainData/output10.pickle done
./TaMillion/TaMillion-Stopword-NOStem-Embedded-TrainData/output11.pickle done
./TaMillion/TaMillion-Stopword-NOStem-Embedded-TrainData/output12.pickle done
./TaMillion/TaMillion-Stopword-NOStem-Embedded-TrainData/output13.pickle 

In [29]:
# Loading Embedded Input Data from Disk

output = []

for i in range(30):
    file_name = "./TaMillion/TaMillion-Stopword-NOStem-Embedded-TrainData/output" + str(i + 1) + ".pickle"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

X = output
output = []

./TaMillion/TaMillion-Stopword-NOStem-Embedded-TrainData/output1.pickle done
./TaMillion/TaMillion-Stopword-NOStem-Embedded-TrainData/output2.pickle done
./TaMillion/TaMillion-Stopword-NOStem-Embedded-TrainData/output3.pickle done
./TaMillion/TaMillion-Stopword-NOStem-Embedded-TrainData/output4.pickle done
./TaMillion/TaMillion-Stopword-NOStem-Embedded-TrainData/output5.pickle done
./TaMillion/TaMillion-Stopword-NOStem-Embedded-TrainData/output6.pickle done
./TaMillion/TaMillion-Stopword-NOStem-Embedded-TrainData/output7.pickle done
./TaMillion/TaMillion-Stopword-NOStem-Embedded-TrainData/output8.pickle done
./TaMillion/TaMillion-Stopword-NOStem-Embedded-TrainData/output9.pickle done
./TaMillion/TaMillion-Stopword-NOStem-Embedded-TrainData/output10.pickle done
./TaMillion/TaMillion-Stopword-NOStem-Embedded-TrainData/output11.pickle done
./TaMillion/TaMillion-Stopword-NOStem-Embedded-TrainData/output12.pickle done
./TaMillion/TaMillion-Stopword-NOStem-Embedded-TrainData/output13.pickle 

In [30]:
X = np.array(X)
X.shape

(5877, 1, 768)

In [31]:
X = X.reshape(5877, 768)
X.shape

(5877, 768)

In [32]:
y = label
y.shape

(5877,)

## **Train Test Split**

In [33]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
X_train.shape

(4701, 768)

In [35]:
X_val.shape

(1176, 768)

## **Logistic Regression**

In [36]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_val)
print(classification_report(y_val, lr_pred))

              precision    recall  f1-score   support

           0       0.81      1.00      0.89       950
           1       0.00      0.00      0.00       226

    accuracy                           0.81      1176
   macro avg       0.40      0.50      0.45      1176
weighted avg       0.65      0.81      0.72      1176



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **Support Vector Machine**

In [37]:
svm = SVC()
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_val)
print(classification_report(y_val, svm_pred))

              precision    recall  f1-score   support

           0       0.81      1.00      0.89       950
           1       0.00      0.00      0.00       226

    accuracy                           0.81      1176
   macro avg       0.40      0.50      0.45      1176
weighted avg       0.65      0.81      0.72      1176



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **Naive Bayes**

In [38]:
nb = GaussianNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_val)
print(classification_report(y_val, nb_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       950
           1       0.19      1.00      0.32       226

    accuracy                           0.19      1176
   macro avg       0.10      0.50      0.16      1176
weighted avg       0.04      0.19      0.06      1176



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **Stochastic Gradient Descent**

In [39]:
sgd = SGDClassifier()
sgd.fit(X_train, y_train)
sgd_pred = sgd.predict(X_val)
print(classification_report(y_val, sgd_pred))

              precision    recall  f1-score   support

           0       0.81      1.00      0.89       950
           1       0.00      0.00      0.00       226

    accuracy                           0.81      1176
   macro avg       0.40      0.50      0.45      1176
weighted avg       0.65      0.81      0.72      1176



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **K Nearest Neighbours**

In [40]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_val)
print(classification_report(y_val, knn_pred))

              precision    recall  f1-score   support

           0       0.81      0.93      0.87       950
           1       0.23      0.08      0.12       226

    accuracy                           0.77      1176
   macro avg       0.52      0.51      0.49      1176
weighted avg       0.70      0.77      0.72      1176



## **Decision tree**

In [41]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_val)
print(classification_report(y_val, dt_pred))

              precision    recall  f1-score   support

           0       0.81      0.91      0.86       950
           1       0.19      0.08      0.12       226

    accuracy                           0.76      1176
   macro avg       0.50      0.50      0.49      1176
weighted avg       0.69      0.76      0.72      1176



## **Random Forest**

In [42]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_val)
print(classification_report(y_val, rf_pred))

              precision    recall  f1-score   support

           0       0.81      1.00      0.89       950
           1       0.00      0.00      0.00       226

    accuracy                           0.81      1176
   macro avg       0.40      0.50      0.45      1176
weighted avg       0.65      0.81      0.72      1176



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **Majority Voting**

In [43]:
mv_pred = []

for i in range(len(lr_pred)):
    one = 0
    zero = 0
    predictions = [rf_pred[i], knn_pred[i], dt_pred[i], lr_pred[i], svm_pred[i], sgd_pred[i], nb_pred[i]]
    for pred in predictions:
        if pred == 1: one += 1
        if pred == 0: zero +=1
    if one > zero: mv_pred.append(1)
    else: mv_pred.append(0)

mv_pred = np.array(mv_pred)

print(classification_report(y_val, mv_pred))

              precision    recall  f1-score   support

           0       0.81      1.00      0.89       950
           1       0.00      0.00      0.00       226

    accuracy                           0.81      1176
   macro avg       0.40      0.50      0.45      1176
weighted avg       0.65      0.81      0.72      1176



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **Loading Testing Data**

In [44]:
dataframe_test = pd.read_csv(
    "Dataset/tam_offesive_withoutlabels_test.tsv", 
    sep='\t'
)
dataframe_test.head()

,id,text
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...


In [45]:
dataframe_test.describe()

,id,text
count,654,654
unique,654,654
top,tam_test_262,பெண் பிள்ளைகளைப் பெற்ற அனைத்து தகப்பன்மார்களுக...
freq,1,1


In [46]:
text_test = dataframe_test['text']

## **Test Data Preprocessing**

In [47]:
text_test = text_test.str.replace(r"[+/#@&*$%:]",'')
text_test = text_test.to_numpy()

In [48]:
text_test = [stopwords_remove(s) for s in text_test]
for i in range(len(text_test)):
  text_test[i] = (" ".join(text_test[i]))

## **Test Feature Extraction**

In [49]:
# Load Transformer Model

tokenizer = AutoTokenizer.from_pretrained("monsoon-nlp/tamillion")
model = AutoModel.from_pretrained("monsoon-nlp/tamillion")

Some weights of the model checkpoint at monsoon-nlp/tamillion were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [50]:
# Sample Output

tokenized_input = tokenizer(
        text_test[0],
        padding=True,
        truncation=False,
        return_tensors='pt'
    )

sample_output = model(**tokenized_input)

In [51]:
print(tokenized_input)

{'input_ids': tensor([[    3,  8689,  1017, 28805,  1085,  2008,  1961,  5222,  3230, 11190,
         14066,  1840, 33480,  8689,  1017,  7187,  8372,  1054,    71,  1041,
          1073, 30637,     4]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [52]:
print(sample_output)

BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[[-8.7158e-04, -1.5213e-04,  1.6477e+00,  ..., -1.5470e-03,
          -2.9372e-05,  9.1286e-05],
         [-8.7158e-04, -1.5213e-04,  1.6477e+00,  ..., -1.5470e-03,
          -2.9372e-05,  9.1286e-05],
         [-8.7158e-04, -1.5213e-04,  1.6477e+00,  ..., -1.5470e-03,
          -2.9372e-05,  9.1286e-05],
         ...,
         [-8.7158e-04, -1.5213e-04,  1.6477e+00,  ..., -1.5470e-03,
          -2.9372e-05,  9.1286e-05],
         [-8.7158e-04, -1.5213e-04,  1.6477e+00,  ..., -1.5470e-03,
          -2.9372e-05,  9.1286e-05],
         [-8.7158e-04, -1.5213e-04,  1.6477e+00,  ..., -1.5470e-03,
          -2.9372e-05,  9.1286e-05]]], grad_fn=<NativeLayerNormBackward0>), past_key_values=None, hidden_states=None, attentions=None, cross_attentions=None)


In [53]:
print(sample_output.last_hidden_state.cpu().detach().numpy().shape)

(1, 23, 768)


In [54]:
# Tokenizing Test Input Data

input = []

for i in text_test:
    tokenized_input = tokenizer(
        i,
        padding=True,
        truncation=False,
        return_tensors='pt'
    )
    input.append(tokenized_input)

In [55]:
# Embedding the Test Input Data

output = []

j = 1

for i in range(len(input)):
    
    model_output = model(**input[i])
    model_output = model_output.last_hidden_state.cpu().detach().numpy()
    output.append(max_pool(model_output))
    
    # Saving Embedded Test Input Data to Disk

    if ((i + 1) % 200) == 0:
        
        file_name = "./TaMillion/TaMillion-Stopword-NOStem-Embedded-TestData/output" + str(j) + ".pickle"
        
        with open(file_name, "wb") as fp:   
            # Pickling
            pickle.dump(output, fp)
        
        print(file_name + " done")
        output = []
        j += 1
        
file_name = "./TaMillion/TaMillion-Stopword-NOStem-Embedded-TestData/output" + str(j) + ".pickle"

with open(file_name, "wb") as fp:   
    # Pickling
    pickle.dump(output, fp)

print(file_name + " done")
output = []

./TaMillion/TaMillion-Stopword-NOStem-Embedded-TestData/output1.pickle done
./TaMillion/TaMillion-Stopword-NOStem-Embedded-TestData/output2.pickle done
./TaMillion/TaMillion-Stopword-NOStem-Embedded-TestData/output3.pickle done
./TaMillion/TaMillion-Stopword-NOStem-Embedded-TestData/output4.pickle done


In [56]:
# Loading Embedded Input Test Data from Disk

output = []

for i in range(4):
    file_name = "./TaMillion/TaMillion-Stopword-NOStem-Embedded-TestData/output" + str(i + 1) + ".pickle"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

X_test = output
output = []

./TaMillion/TaMillion-Stopword-NOStem-Embedded-TestData/output1.pickle done
./TaMillion/TaMillion-Stopword-NOStem-Embedded-TestData/output2.pickle done
./TaMillion/TaMillion-Stopword-NOStem-Embedded-TestData/output3.pickle done
./TaMillion/TaMillion-Stopword-NOStem-Embedded-TestData/output4.pickle done


In [57]:
X_test = np.array(X_test)
X_test.shape

(654, 1, 768)

In [58]:
X_test = X_test.reshape(654, 768)
X_test.shape

(654, 768)

## **Getting Predictions**

In [59]:
# Logistic Regression
lr_test_pred = lr.predict(X_test)

# Support Vector Machine
svm_test_pred = svm.predict(X_test)

# Naive Bayes
nb_test_pred = nb.predict(X_test)

# Stochastic Gradient Descent
sgd_test_pred = sgd.predict(X_test)

# K Nearest Neighbours
knn_test_pred = knn.predict(X_test)

# Decision Tree
dt_test_pred = dt.predict(X_test)

# Random Forest
rf_test_pred = rf.predict(X_test)

In [60]:
# Majority Voting
mv_test_pred = []

for i in range(len(lr_test_pred)):
    one = 0
    zero = 0
    predictions = [lr_test_pred[i], svm_test_pred[i], nb_test_pred[i], 
                   sgd_test_pred[i], knn_test_pred[i], dt_test_pred[i], rf_test_pred[i]]
    for pred in predictions:
        if pred == 1: one += 1
        if pred == 0: zero +=1
    if one > zero: mv_test_pred.append(1)
    else: mv_test_pred.append(0)

mv_test_pred = np.array(mv_test_pred)

## **Evaluating the Predictions**

In [61]:
dataframe_eval = pd.read_csv(
    "Dataset/tamil_offensive_test task 1 with labels.tsv", 
    sep='\t'
)

In [62]:
y_test = dataframe_eval['category']
y_test = LabelEncoder().fit_transform(y_test)

In [63]:
print('Logistic Regression')
print(classification_report(y_test, lr_test_pred))

Logistic Regression
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       536
           1       0.00      0.00      0.00       118

    accuracy                           0.82       654
   macro avg       0.41      0.50      0.45       654
weighted avg       0.67      0.82      0.74       654



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [64]:
print('Support Vector Machine')
print(classification_report(y_test, svm_test_pred))

Support Vector Machine
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       536
           1       0.00      0.00      0.00       118

    accuracy                           0.82       654
   macro avg       0.41      0.50      0.45       654
weighted avg       0.67      0.82      0.74       654



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [65]:
print('Naive Bayes')
print(classification_report(y_test, nb_test_pred))

Naive Bayes
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       536
           1       0.18      1.00      0.31       118

    accuracy                           0.18       654
   macro avg       0.09      0.50      0.15       654
weighted avg       0.03      0.18      0.06       654



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [66]:
print('Stochastic Gradient Descent')
print(classification_report(y_test, sgd_test_pred))

Stochastic Gradient Descent
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       536
           1       0.00      0.00      0.00       118

    accuracy                           0.82       654
   macro avg       0.41      0.50      0.45       654
weighted avg       0.67      0.82      0.74       654



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [67]:
print('K Nearest Neighbours')
print(classification_report(y_test, knn_test_pred))

K Nearest Neighbours
              precision    recall  f1-score   support

           0       0.82      0.94      0.88       536
           1       0.21      0.07      0.10       118

    accuracy                           0.78       654
   macro avg       0.51      0.50      0.49       654
weighted avg       0.71      0.78      0.74       654



In [68]:
print('Decision Tree')
print(classification_report(y_test, dt_test_pred))

Decision Tree
              precision    recall  f1-score   support

           0       0.82      0.91      0.87       536
           1       0.23      0.12      0.16       118

    accuracy                           0.77       654
   macro avg       0.53      0.52      0.51       654
weighted avg       0.72      0.77      0.74       654



In [69]:
print('Random Forest')
print(classification_report(y_test, rf_test_pred))

Random Forest
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       536
           1       0.00      0.00      0.00       118

    accuracy                           0.82       654
   macro avg       0.41      0.50      0.45       654
weighted avg       0.67      0.82      0.74       654



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [70]:
print('Majority Voting')
print(classification_report(y_test, mv_test_pred))

Majority Voting
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       536
           1       0.00      0.00      0.00       118

    accuracy                           0.82       654
   macro avg       0.41      0.50      0.45       654
weighted avg       0.67      0.82      0.74       654



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
